In [ ]:
import requests
import json # Menggunakan json

# --- KONFIGURASI ---
# API key
API_KEY = "242819b7dfe84b4488a34756251809" 
LOKASI_CONTOH = "Malang" # Coba ambil data untuk Malang
# --- AKHIR KONFIGURASI --

# URL endpoint dari WeatherAPI
url = f"http://api.weatherapi.com/v1/current.json?key={API_KEY}&q={LOKASI_CONTOH}&aqi=no"

print(f"Mengambil data cuaca untuk: {LOKASI_CONTOH}...")

try:
    response = requests.get(url, timeout=10)
    response.raise_for_status()  # Error jika status code bukan 2xx
    
    data = response.json()
    
    # Cetak seluruh response JSON dengan format yang rapi
    print("\n--- Struktur Data JSON Lengkap ---")
    print(json.dumps(data, indent=4))
    
except requests.exceptions.RequestException as e:
    print(f"Gagal mendapatkan data: {e}")

Mengambil data cuaca untuk: Malang...

--- Struktur Data JSON Lengkap ---
{
    "location": {
        "name": "Malang",
        "region": "East Java",
        "country": "Indonesia",
        "lat": -7.9797,
        "lon": 112.6304,
        "tz_id": "Asia/Jakarta",
        "localtime_epoch": 1758453324,
        "localtime": "2025-09-21 18:15"
    },
    "current": {
        "last_updated_epoch": 1758453300,
        "last_updated": "2025-09-21 18:15",
        "temp_c": 30.1,
        "temp_f": 86.2,
        "is_day": 0,
        "condition": {
            "text": "Light rain shower",
            "icon": "//cdn.weatherapi.com/weather/64x64/night/353.png",
            "code": 1240
        },
        "wind_mph": 2.2,
        "wind_kph": 3.6,
        "wind_degree": 226,
        "wind_dir": "SW",
        "pressure_mb": 1009.0,
        "pressure_in": 29.8,
        "precip_mm": 0.34,
        "precip_in": 0.01,
        "humidity": 75,
        "cloud": 25,
        "feelslike_c": 38.3,
        "feel

In [ ]:
# --- 1. IMPORT LIBRARY YANG DIBUTUHKAN ---
import pandas as pd  # Untuk membaca dan menulis file Excel
import requests      # Untuk mengirim permintaan ke API cuaca
import threading     # Untuk menjalankan banyak permintaan API secara bersamaan
from datetime import datetime # Untuk menghitung waktu eksekusi

# --- 2. KONFIGURASI UTAMA ---
# API key
API_KEY = "242819b7dfe84b4488a34756251809"
# Nama file Excel yang berisi daftar kecamatan
NAMA_FILE_EXCEL = "DaftarKecamatan_kelompok_6.xlsx"
# Nama kolom di file Excel yang akan dibaca
NAMA_KOLOM_KECAMATAN = "Kecamatan"

# --- 3. PERSIAPAN UNTUK MULTI-THREADING ---
# 'results' adalah list kosong untuk menyimpan data cuaca yang berhasil diambil dari setiap thread.
results = []
# 'lock' digunakan untuk memastikan tidak ada dua thread yang menulis ke list 'results' di waktu yang bersamaan (Race).
# Ini mencegah data menjadi korup atau hilang.
lock = threading.Lock()

# --- 4. FUNGSI UNTUK MENGAMBIL DATA CUACA (UNTUK SETIAP THREAD) ---
def get_weather_data(kecamatan, index):
    """
    Fungsi ini dijalankan oleh setiap thread. Satu thread untuk satu kecamatan.
    'kecamatan' adalah nama lokasi (misal: "Waru").
    'index' adalah nomor baris asli dari Excel agar data tidak tertukar.
    """
    # URL API diformat dengan API Key dan nama kecamatan yang spesifik
    url = f"http://api.weatherapi.com/v1/current.json?key={API_KEY}&q={kecamatan}&aqi=no"
    
    try:
        # Kirim permintaan GET ke URL. Timeout 10 detik.
        response = requests.get(url, timeout=10)
        # Jika status respons bukan 200 OK (misal: 404 Not Found), maka akan muncul error
        response.raise_for_status()
        # Ubah respons dari format JSON menjadi Python dictionary
        data = response.json()

        # Buat dictionary baru hanya dengan data yang kita butuhkan
        weather_info = {
            'Last Update': data['location']['localtime'],
            'Suhu (°C)': data['current']['temp_c'],
            'Kelembapan (%)': data['current']['humidity'],
            'Kondisi Cuaca': data['current']['condition']['text'],
            'Kecepatan Angin (km/h)': data['current']['wind_kph'],
            'Arah Angin': data['current']['wind_dir'],
            'Sinar UV': data['current']['uv']
        }
        print(f"✅ Berhasil -> {kecamatan}")

    except requests.exceptions.RequestException:
        # Jika terjadi error apapun (lokasi tidak ada, koneksi gagal, dll.)
        print(f"Gagal -> {kecamatan}")
        # Buat dictionary berisi pesan 'Error' untuk menandai kegagalan
        weather_info = {
            'Last Update': 'Error',
            'Suhu (°C)': 'Error',
            'Kelembapan (%)': 'Error',
            'Kondisi Cuaca': 'Error',
            'Kecepatan Angin (km/h)': 'Error', 
            'Arah Angin' : 'Error',
            'Sinar UV': 'Error'
        }

    # Sebelum menyimpan hasil, 'kunci' dulu list 'results'
    with lock:
        # Simpan hasil (baik data maupun error) bersama dengan nomor baris aslinya
        results.append({'index': index, 'data': weather_info})

# --- 5. FUNGSI UTAMA UNTUK MENJALANKAN SEMUANYA ---
def main():
    # Catat waktu mulai
    start_time = datetime.now()

    # Baca file Excel dan simpan datanya ke dalam sebuah DataFrame pandas
    df = pd.read_excel(NAMA_FILE_EXCEL, engine='openpyxl')
    
    # Buat list kosong untuk menampung semua thread yang akan dibuat
    threads = []
    
    print(f"Memulai pengambilan data untuk {len(df)} kecamatan...")

    # Looping untuk setiap baris di dalam DataFrame Excel
    for index, row in df.iterrows():
        # Ambil nama kecamatan dari kolom yang sudah ditentukan
        kecamatan = row[NAMA_KOLOM_KECAMATAN]
        # Jika nama kecamatan tidak kosong
        if pd.notna(kecamatan):
            # Buat thread baru. Targetnya adalah fungsi 'get_weather_data'.
            # Argumennya adalah nama kecamatan dan nomor barisnya.
            thread = threading.Thread(target=get_weather_data, args=(kecamatan, index))
            # Tambahkan thread yang baru dibuat ke dalam list 'threads'
            threads.append(thread)
            # Mulai jalankan thread
            thread.start()

    # Setelah semua thread dibuat dan dimulai, tunggu sampai semuanya selesai bekerja
    for thread in threads:
        thread.join()

    print("\nSemua thread selesai. Menulis data ke Excel...")

    # Loop melalui hasil yang sudah terkumpul dari semua thread
    for result in results:
        # Ambil nomor baris asli
        idx = result['index']
        # Loop melalui setiap item data (misal: 'Suhu (°C)': 30)
        for key, value in result['data'].items():
            # Masukkan nilai ke DataFrame di baris (idx) dan kolom (key) yang sesuai
            df.loc[idx, key] = value
            
    # Tulis ulang seluruh DataFrame yang sudah diperbarui ke file Excel yang sama
    # index=False berarti nomor baris dari pandas tidak akan ikut ditulis ke Excel
    df.to_excel(NAMA_FILE_EXCEL, index=False)
    
    # Catat waktu selesai
    end_time = datetime.now()
    
    print(f"\n✅ Selesai! Data cuaca telah ditulis ke '{NAMA_FILE_EXCEL}'.")
    print(f"Total waktu eksekusi: {end_time - start_time}")

# --- 6. MEMANGGIL FUNGSI UTAMA UNTUK MEMULAI SKRIP ---
if __name__ == "__main__":
    main()

Memulai pengambilan data untuk 591 kecamatan...
✅ Berhasil -> Benowo, Surabaya
✅ Berhasil -> Bulak, Surabaya
✅ Berhasil -> Bubutan, Surabaya
✅ Berhasil -> Semampir, Surabaya
✅ Berhasil -> Tegalsari, Surabaya
✅ Berhasil -> Tandes, Surabaya
✅ Berhasil -> Asemrowo, Surabaya
✅ Berhasil -> Sambikerep, Surabaya
✅ Berhasil -> Dukuh Pakis, Surabaya
✅ Berhasil -> Suko Manunggal, Surabaya
✅ Berhasil -> Simokerto, Surabaya
✅ Berhasil -> Wonocolo, Surabaya
✅ Berhasil -> Wonokromo, Surabaya
✅ Berhasil -> Candi, Sidoarjo
✅ Berhasil -> Tenggilis Mejoyo, Surabaya
✅ Berhasil -> Gedangan, Sidoarjo
✅ Berhasil -> Buduran, Sidoarjo
✅ Berhasil -> Porong, Sidoarjo
✅ Berhasil -> Taman, Sidoarjo
✅ Berhasil -> Tanggulangin, Sidoarjo
✅ Berhasil -> Sedati, Sidoarjo
✅ Berhasil -> Balong, Ponorogo
✅ Berhasil -> Prambon, Sidoarjo
✅ Berhasil -> Waru, Sidoarjo
✅ Berhasil -> Tarik, Sidoarjo
✅ Berhasil -> Sawahan, Surabaya
✅ Berhasil -> Rungkut, Surabaya
✅ Berhasil -> Babadan, Ponorogo
✅ Berhasil -> Badegan, Ponorogo
✅ 

C:\Users\peter\AppData\Local\Temp\ipykernel_10016\3817088020.py:112: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Error' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[idx, key] = value
C:\Users\peter\AppData\Local\Temp\ipykernel_10016\3817088020.py:112: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Error' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[idx, key] = value
C:\Users\peter\AppData\Local\Temp\ipykernel_10016\3817088020.py:112: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Error' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[idx, key] = value
C:\Users\peter\AppData\Local\Temp\ipyke


✅ Selesai! Data cuaca telah ditulis ke 'DaftarKecamatan.xlsx'.
Total waktu eksekusi: 0:00:03.443390
